This is a test for CIS2M.

In [136]:
# def mcisCF(Ac,Bc,Gc,F)
import numpy as np

Ac = np.array([[1,2],[2,3]])
Bc = np.array([1,2])
Gc = np.array([[1,-2],[3,-5],[-7,8]])
F = Bc

n = Ac.shape[0] # number of original variables
m = Gc.shape[0] # number of original inequalities

## Construct MCIS in High Dimension
print('Computing MCIS in closed-form, in high dimensional space..')

# Construct Ghat0 and Hhat0:
tmpG = np.zeros((m*n,n))
Hhat0 = np.zeros((m,n+m*n))
                
for j in range(m):
    # For Ghat0:
    tmpG[j*n:(j+1)*n,:] = np.diag(Gc[j,:]);
    
    # Hhat0 here --faster than doing sparse.
    for i in range(n):
        Hhat0[j,(j+1)*n+i] = 1;
# tmpG = sparse(tmpG);
# Ghat0 = [tmpG -speye(m*n)];
# Hhat0 = sparse(Hhat0);

Ghat0 = np.concatenate((tmpG,-np.identity(m*n)),axis=1)

# "Index" matrices:
Gsign = np.transpose(Gc)
Gsign = np.concatenate(Gsign)

P = np.zeros((n*m,1))
T = np.zeros((n*m,1))
for i in range(n*m):
    if (Gsign[i]>0):
        P[i] = -1/Gsign[i]
    elif (Gsign[i]<0):
        T[i] = 1/Gsign[i]

Pidx = [i for i, e in enumerate(P) if e != 0]
Tidx = [i for i, e in enumerate(T) if e != 0]
# Tidx = np.nonzero(T)
# Tidx = T != 0

# Construct the combinatorial constraints matrix
sP = len(Pidx)
sT = len(Tidx)
hGamma = sP*sT

# pre-allocate
gammaCell = np.zeros((sP,n+m*n,sT))
# for p in sP:
#     gammaCell{p} = np.zeros((n+m*n,sT))

for p in range(sP):
    for t in range(sT):
        gammaCell[p,n+Pidx[p],t] = P[Pidx[p]]
        gammaCell[p,n+Tidx[t],t] = T[Tidx[t]]
        
Gamma = np.transpose(np.concatenate(gammaCell,axis=1))

# Costruct the passing-through-dynamics-constraints matrix Gtilde
# AcSP = sparse(Ac)
# Ahat = [AcSP sparse(n,n*m); sparse(n*m,n) speye(n*m,n*m)]
# BcSP = sparse(Bc)
# Bhat = [BcSP; sparse(n*m,1)]

Ahat = np.concatenate((np.concatenate((Ac,np.zeros((n,m*n))),axis=1),np.concatenate((np.zeros((m*n,n)),np.identity(m*n)),axis=1)),axis=0)
Bhat = np.concatenate((Bc,np.zeros(m*n)))

# Ridx = indices that involve 'u'.
# tempA1 = Ghat0 Ahat^{1} till Ghat0 Ahat^{n} at each step 'i' without
# indices in Ridx1.
# tempB1 = Ghat0 Ac^{0} Bc till Ghat0 Ac^{n-1} Bc at each step 'i' without
# indices in Ridx1.
# Gtilde = concatinate vertically the above resulting inequalities.

Gtilde = np.empty((0,n+m*n))
tempA1 = Ghat0.dot(Ahat)
tempB1 = Ghat0.dot(Bhat)

for i in range(n):
    Ridx = [i for i, e in enumerate(tempB1) if e != 0]
    
    tempA1 = np.delete(tempA1,Ridx,0)
    Gtilde = np.concatenate((gtildeCell,tempA1))
    
    tempB1 = tempA1.dot(Bhat)
    tempA1 = tempA1.dot(Ahat)

hGtilde = Gtilde.shape[0]
Ftilde = np.zeros(hGtilde)

# MCIS in closed-form:
# [Sum of f vars; Original lift; Constraints by passing through dynamics;
# Constraints on f vars from FME]
mcisA = np.concatenate((Hhat0,Ghat0,Gtilde,Gamma))

# F = sparse(F);

mcisb = np.concatenate((F,np.zeros(m*n),Ftilde,np.zeros(hGamma)))

print('..done!')

Computing MCIS in closed-form, in high dimensional space..
..done!
